In [3]:
%config IPCompleter.use_jedi = False
%pdb off
# %load_ext viztracer
# from viztracer import VizTracer
%load_ext autoreload
%autoreload 3
import sys
from pathlib import Path

# required to enable non-blocking interaction:
%gui qt5

from copy import deepcopy
from numba import jit
import numpy as np
import pandas as pd

# Pho's Formatting Preferences
# from pyphocorehelpers.preferences_helpers import set_pho_preferences, set_pho_preferences_concise, set_pho_preferences_verbose
# set_pho_preferences_concise()

## Pho's Custom Libraries:
from pyphocorehelpers.general_helpers import CodeConversion
from pyphocorehelpers.function_helpers import function_attributes
from pyphocorehelpers.print_helpers import print_keys_if_possible, print_value_overview_only, document_active_variables, objsize, print_object_memory_usage, debug_dump_object_member_shapes, TypePrintMode
from pyphocorehelpers.print_helpers import get_now_day_str, get_now_time_str, get_now_time_precise_str
from pyphocorehelpers.programming_helpers import IPythonHelpers
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path

# NeuroPy (Diba Lab Python Repo) Loading
# from neuropy import core
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.core.epoch import NamedTimerange, Epoch
from neuropy.core.ratemap import Ratemap
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder
from neuropy.core.session.Formats.Specific.BapunDataSessionFormat import BapunDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.RachelDataSessionFormat import RachelDataSessionFormat
from neuropy.core.session.Formats.Specific.HiroDataSessionFormat import HiroDataSessionFormatRegisteredClass

## For computation parameters:
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.utils.dynamic_container import DynamicContainer
from neuropy.utils.result_context import IdentifyingContext
from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths
from neuropy.core.neurons import NeuronType

# pyPhoPlaceCellAnalysis:
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import NeuropyPipeline # get_neuron_identities
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import export_pyqtgraph_plot, create_daily_programmatic_display_function_testing_folder_if_needed, session_context_to_relative_path
from pyphoplacecellanalysis.General.Batch.NonInteractiveWrapper import batch_load_session, batch_extended_computations, SessionBatchProgress, batch_programmatic_figures, batch_extended_programmatic_figures
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import PipelineSavingScheme
from pyphoplacecellanalysis.Pho2D.matplotlib.visualize_heatmap import visualize_heatmap

# from PendingNotebookCode import _perform_batch_plot, _build_batch_plot_kwargs

session_batch_status = {}
session_batch_errors = {}
enable_saving_to_disk = False

global_data_root_parent_path = find_first_extant_path([Path(r'W:\Data'), Path(r'/media/MAX/Data'), Path(r'/Volumes/MoverNew/data'), Path(r'/home/halechr/turbo/Data')])
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"

Automatic pdb calling has been turned OFF
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load Pipeline

In [4]:
# ==================================================================================================================== #
# Load Data                                                                                                            #
# ==================================================================================================================== #

active_data_mode_name = 'kdiba'

## Data must be pre-processed using the MATLAB script located here: 
#     neuropy/data_session_pre_processing_scripts/KDIBA/IIDataMat_Export_ToPython_2022_08_01.m
# From pre-computed .mat files:

local_session_root_parent_context = IdentifyingContext(format_name=active_data_mode_name) # , animal_name='', configuration_name='one', session_name=self.session_name
local_session_root_parent_path = global_data_root_parent_path.joinpath('KDIBA')

good_contexts_list = [IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30')]


In [5]:
%pdb off
# %%viztracer

curr_context = good_contexts_list[0] # select the session from all of the good sessions here.
local_session_parent_path = local_session_root_parent_path.joinpath(curr_context.animal, curr_context.exper_name) # 'gor01', 'one' - probably not needed anymore
basedir = local_session_parent_path.joinpath(curr_context.session_name).resolve()
print(f'basedir: {str(basedir)}')

# # Read if possible:
saving_mode = PipelineSavingScheme.SKIP_SAVING
force_reload = False

# Force write:
# saving_mode = PipelineSavingScheme.OVERWRITE_IN_PLACE # PipelineSavingScheme.OVERWRITE_IN_PLACE
# force_reload = True

# ==================================================================================================================== #
# Load Pipeline                                                                                                        #
# ==================================================================================================================== #
# epoch_name_whitelist = ['maze']
epoch_name_whitelist = None
active_computation_functions_name_whitelist=['_perform_baseline_placefield_computation',
                                        # '_perform_time_dependent_placefield_computation',
                                        '_perform_extended_statistics_computation',
                                        '_perform_position_decoding_computation', 
                                        '_perform_firing_rate_trends_computation',
                                        '_perform_pf_find_ratemap_peaks_computation',
                                        # '_perform_time_dependent_pf_sequential_surprise_computation'
                                        # '_perform_two_step_position_decoding_computation',
                                        # '_perform_recursive_latent_placefield_decoding'
                                    ]
# active_computation_functions_name_whitelist=['_perform_baseline_placefield_computation']

curr_active_pipeline = batch_load_session(global_data_root_parent_path, active_data_mode_name, basedir, epoch_name_whitelist=epoch_name_whitelist,
                                          computation_functions_name_whitelist=active_computation_functions_name_whitelist,
                                          saving_mode=saving_mode, force_reload=force_reload,
                                          skip_extended_batch_computations=True, debug_print=False, fail_on_exception=True)

# 6m 1.1s


Automatic pdb calling has been turned OFF
basedir: W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15
Loading loaded session pickle file results : W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\loadedSessPickle.pkl... done.
Loading pickled pipeline success: W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\loadedSessPickle.pkl.
property already present in pickled version. No need to save.
using provided computation_functions_name_whitelist: ['_perform_baseline_placefield_computation', '_perform_extended_statistics_computation', '_perform_position_decoding_computation', '_perform_firing_rate_trends_computation', '_perform_pf_find_ratemap_peaks_computation']
	 TODO: this will prevent recomputation even when the blacklist/whitelist or computation function definitions change. Rework so that this is smarter.
updating computation_results...
done.
	 TODO: this will prevent recomputation even when the blacklist/whitelist or computation function definitions change. Rework so that this is smarter.
updating computa

In [6]:
## Post Compute Validate 2023-05-16:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import LongShortPipelineTests

def post_compute_validate(curr_active_pipeline):
	""" 2023-05-16 - Ensures that the laps are used for the placefield computation epochs, the number of bins are the same between the long and short tracks. """
	LongShortPipelineTests(curr_active_pipeline=curr_active_pipeline).validate()

post_compute_validate(curr_active_pipeline)

In [7]:
long_epoch_name, short_epoch_name, global_epoch_name = curr_active_pipeline.find_LongShortGlobal_epoch_names()
long_session, short_session, global_session = [curr_active_pipeline.filtered_sessions[an_epoch_name] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_results, short_results, global_results = [curr_active_pipeline.computation_results[an_epoch_name]['computed_data'] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_pf1D, short_pf1D, global_pf1D = long_results.pf1D, short_results.pf1D, global_results.pf1D
long_pf2D, short_pf2D, global_pf2D = long_results.pf2D, short_results.pf2D, global_results.pf2D

# Get existing laps from session:
long_laps, short_laps, global_laps = [curr_active_pipeline.filtered_sessions[an_epoch_name].laps.as_epoch_obj() for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_replays, short_replays, global_replays = [Epoch(curr_active_pipeline.filtered_sessions[an_epoch_name].replay.epochs.get_valid_df()) for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
# short_laps.n_epochs: 40, long_laps.n_epochs: 40
# short_replays.n_epochs: 6, long_replays.n_epochs: 8

print(f'short_laps.n_epochs: {short_laps.n_epochs}, long_laps.n_epochs: {long_laps.n_epochs}')
print(f'short_replays.n_epochs: {short_replays.n_epochs}, long_replays.n_epochs: {long_replays.n_epochs}')

short_laps.n_epochs: 40, long_laps.n_epochs: 40
short_replays.n_epochs: 181, long_replays.n_epochs: 222


In [8]:
# , '_perform_position_decoding_computation'
curr_active_pipeline.perform_specific_computation(computation_functions_name_whitelist=['_perform_extended_statistics_computation', '_perform_firing_rate_trends_computation', '_perform_pf_find_ratemap_peaks_computation'],
                                                  fail_on_exception=True, debug_print=False) # fail_on_exception MUST be True or error handling is all messed up

Performing run_specific_computations_single_context on filtered_session with filter named "maze1"...
Performing run_specific_computations_single_context on filtered_session with filter named "maze2"...
Performing run_specific_computations_single_context on filtered_session with filter named "maze"...


In [10]:

# 2023-01-* - Call extended computations to build `_display_short_long_firing_rate_index_comparison` figures:
extended_computations_include_whitelist=['long_short_fr_indicies_analyses', 'jonathan_firing_rate_analysis', 'long_short_decoding_analyses'] # do only specifiedl
# extended_computations_include_whitelist=['long_short_fr_indicies_analyses', 'jonathan_firing_rate_analysis'] # do only specifiedl
newly_computed_values = batch_extended_computations(curr_active_pipeline, include_whitelist=extended_computations_include_whitelist, include_global_functions=True, fail_on_exception=True, progress_print=True, force_recompute=True, debug_print=False)
curr_active_pipeline.save_global_computation_results()

included whitelist is specified: ['long_short_fr_indicies_analyses', 'jonathan_firing_rate_analysis', 'long_short_decoding_analyses'], so only performing these extended computations.
Running batch_extended_computations(...) with global_epoch_name: "maze"
jonathan_firing_rate_analysis missing.
	 Recomputing jonathan_firing_rate_analysis...
	 done.
long_short_fr_indicies_analyses missing.
	 Recomputing long_short_fr_indicies_analyses...
	 done.
long_short_decoding_analyses missing.
	 Recomputing long_short_decoding_analyses...
sane_midpoint_x: 119.48022705218361, hardcoded_track_midpoint_x: None, track_min_max_x: (22.736279243974774, 261.696733348342)
hardcoded_track_midpoint_x is None, falling back to sane_midpoint_x... 119.48022705218361
desc_crossings_x: (20,), asc_crossings_x: (20,)
setting new computation epochs because laps changed.
using self.config.grid_bin_bounds_1D: (22.736279243974774, 261.696733348342)
using self.config.grid_bin_bounds: ((22.736279243974774, 261.696733348342)

In [6]:
curr_active_pipeline.save_pipeline(saving_mode=PipelineSavingScheme.TEMP_THEN_OVERWRITE) # AttributeError: 'PfND_TimeDependent' object has no attribute '_included_thresh_neurons_indx'
# TypeError: cannot pickle 'MplMultiTab' object

finalized_loaded_sess_pickle_path: W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\loadedSessPickle.pkl
Saving (file mode 'w+b') saved session pickle file results : W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\20230519233846-loadedSessPickle.pkl... done.
moving new output at 'W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\20230519233846-loadedSessPickle.pkl' -> to desired location: 'W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\loadedSessPickle.pkl'


WindowsPath('W:/Data/KDIBA/gor01/one/2006-6-08_14-26-15/loadedSessPickle.pkl')

In [ ]:
curr_active_pipeline.save_pipeline(saving_mode=PipelineSavingScheme.OVERWRITE_IN_PLACE)

In [ ]:
curr_active_pipeline.save_global_computation_results()

In [ ]:
# curr_active_pipeline.load_pickled_global_computation_results()

# End Run

In [ ]:
## TODO 2025-05-16 - These computation_kwargs_lists don't get passed in?
# time_bin_size=0.025
decoding_time_bin_size=0.02
curr_active_pipeline.perform_specific_computation(computation_functions_name_whitelist=['_perform_long_short_decoding_analyses'], computation_kwargs_list=[dict(decoding_time_bin_size=decoding_time_bin_size, perform_cache_load=False)], fail_on_exception=True, debug_print=False) # fail_on_exception MUST be True or error handling is all messed up 

# , decoding_time_bin_size = 0.02

In [11]:
## long_short_decoding_analyses:
curr_long_short_decoding_analyses = curr_active_pipeline.global_computation_results.computed_data['long_short_leave_one_out_decoding_analysis']
## Extract variables from results object:
long_one_step_decoder_1D, short_one_step_decoder_1D, long_replays, short_replays, global_replays, long_shared_aclus_only_decoder, short_shared_aclus_only_decoder, shared_aclus, long_short_pf_neurons_diff, n_neurons, long_results_obj, short_results_obj, is_global = curr_long_short_decoding_analyses.long_decoder, curr_long_short_decoding_analyses.short_decoder, curr_long_short_decoding_analyses.long_replays, curr_long_short_decoding_analyses.short_replays, curr_long_short_decoding_analyses.global_replays, curr_long_short_decoding_analyses.long_shared_aclus_only_decoder, curr_long_short_decoding_analyses.short_shared_aclus_only_decoder, curr_long_short_decoding_analyses.shared_aclus, curr_long_short_decoding_analyses.long_short_pf_neurons_diff, curr_long_short_decoding_analyses.n_neurons, curr_long_short_decoding_analyses.long_results_obj, curr_long_short_decoding_analyses.short_results_obj, curr_long_short_decoding_analyses.is_global


In [16]:
# (long_one_step_decoder_1D, short_one_step_decoder_1D), (long_one_step_decoder_2D, short_one_step_decoder_2D) = compute_short_long_constrained_decoders(curr_active_pipeline, recalculate_anyway=True)
long_epoch_name, short_epoch_name, global_epoch_name = curr_active_pipeline.find_LongShortGlobal_epoch_names()
long_epoch_context, short_epoch_context, global_epoch_context = [curr_active_pipeline.filtered_contexts[a_name] for a_name in (long_epoch_name, short_epoch_name, global_epoch_name)]
long_session, short_session, global_session = [curr_active_pipeline.filtered_sessions[an_epoch_name] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_results, short_results, global_results = [curr_active_pipeline.computation_results[an_epoch_name]['computed_data'] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_pf1D, short_pf1D, global_pf1D = long_results.pf1D, short_results.pf1D, global_results.pf1D
long_pf2D, short_pf2D, global_pf2D = long_results.pf2D, short_results.pf2D, global_results.pf2D
decoding_time_bin_size = long_one_step_decoder_1D.time_bin_size # 1.0/30.0 # 0.03333333333333333

In [ ]:
global_session.get_output_path()

In [ ]:
long_results_obj.active_filter_epochs

In [ ]:
curr_active_pipeline.global_computation_results.computed_data['long_short_fr_indicies_analysis'].long_mean_replays_frs
curr_active_pipeline.global_computation_results.computed_data['long_short_fr_indicies_analysis'].short_mean_replays_frs

# curr_active_pipeline.global_computation_results.computed_data.long_short_fr_indicies_analysis


In [13]:
from neuropy.core.neurons import NeuronType

# rate_remapping_df.to_clipboard(index=False)
# rate_remapping_df.to_csv(f'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/data/rate_remapping_df.csv', index=False)

# rate_remapping_df = pd.read_csv(f'C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/data/rate_remapping_df.csv')
# rate_remapping_df

from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import compute_rate_remapping_stats
rate_remapping_df = compute_rate_remapping_stats(curr_active_pipeline.global_computation_results.computed_data.long_short_fr_indicies_analysis, global_session.neurons.aclu_to_neuron_type_map, considerable_remapping_threshold=0.7)
rate_remapping_df

,neuron_type,laps,replays,skew,max_axis_distance_from_center,distance_from_center,has_considerable_remapping,render_color
aclu,,,,,,,,


In [15]:
high_remapping_cells_only = rate_remapping_df[rate_remapping_df['has_considerable_remapping']] 
high_remapping_cells_only

,neuron_type,laps,replays,skew,max_axis_distance_from_center,distance_from_center,has_considerable_remapping,render_color
aclu,,,,,,,,


In [ ]:
# rate_remapping_df.to_clipboard()
rate_remapping_df

In [19]:
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import plot_rr_aclu

rr_aclus = np.array(list(curr_active_pipeline.global_computation_results.computed_data.long_short_fr_indicies_analysis.y_frs_index.keys()))
rr_neuron_type = [global_session.neurons.aclu_to_neuron_type_map[aclu] for aclu in rr_aclus]
rr_laps = np.array(list(curr_active_pipeline.global_computation_results.computed_data.long_short_fr_indicies_analysis.y_frs_index.values()))
rr_replays = np.array(list(curr_active_pipeline.global_computation_results.computed_data.long_short_fr_indicies_analysis.x_frs_index.values()))
rr_skew = rr_laps / rr_replays

n_debug_limit = 100
fig, ax = plot_rr_aclu([str(aclu) for aclu in rr_aclus[:n_debug_limit]], rr_laps=rr_laps[:n_debug_limit], rr_replays=rr_replays[:n_debug_limit])

fig, axs, sort_indicies = plot_rr_aclu([str(aclu) for aclu in curr_page_rr_aclus], rr_laps=curr_page_rr_laps, rr_replays=curr_page_rr_replays, rr_neuron_types=curr_page_rr_neuron_type, fig=self.plots.fig, axs=self.plots.axs)

TypeError: plot_rr_aclu() missing 1 required positional argument: 'rr_neuron_types'

In [20]:
# Extract rr_* variables from rate_remapping_df
rr_aclus = rate_remapping_df.index.values
rr_laps, rr_replays, rr_skew, rr_neuron_type = [rate_remapping_df[n].values for n in ['laps', 'replays', 'skew', 'neuron_type']]


In [ ]:
# Plot using Paginator:
import matplotlib.pyplot as plt
%matplotlib qt

from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import RateRemappingPaginatedFigureController

## Paginated multi-plot
active_identifying_session_ctx = curr_active_pipeline.sess.get_context()
_out_rr_pagination_controller = RateRemappingPaginatedFigureController.init_from_rr_data(rr_aclus, rr_laps, rr_replays, rr_neuron_type, max_subplots_per_page=20, a_name='TestRateRemappingPaginatedFigureController', active_context=active_identifying_session_ctx)
a_paginator = _out_rr_pagination_controller.plots_data.paginator

In [ ]:

# Set window icon to make it distinguishable in task bar:
from silx.gui.icons import getQIcon
_out_rr_pagination_controller.ui.mw.setWindowIcon(getQIcon('plot-window')) # QtGui.QIcon('icon.png')


In [ ]:
curr_active_pipeline.global_computation_results

# Weird Plotting Stuff

In [ ]:
_plot_obj = curr_active_pipeline.plot
_plot_obj._display_spike_rasters_window()
# _plot_obj

In [ ]:
curr_active_pipeline.reload_default_display_functions()
curr_active_pipeline.plot._display_jonathan_interactive_replay_firing_rate_comparison


In [ ]:
curr_active_pipeline.plot._display_batch_pho_jonathan_replay_firing_rate_comparison()


In [ ]:
curr_active_pipeline.plot._display_short_long_firing_rate_index_comparison()


In [ ]:
active_identifying_session_ctx, active_session_figures_out_path, active_out_figures_list = batch_programmatic_figures(curr_active_pipeline)

In [ ]:
# import pyflakes
# Fixed but requires 'pyflakes' for code editor, which I removed. ModuleNotFoundError: No module named 'pyflakes'
active_identifying_session_ctx = curr_active_pipeline.sess.get_context() # 'bapun_RatN_Day4_2019-10-15_11-30-06'
display_output = curr_active_pipeline.display('_display_context_nested_docks', active_identifying_session_ctx, enable_gui=True, debug_print=False) # returns {'master_dock_win': master_dock_win, 'app': app, 'out_items': out_items}
master_dock_win = display_output['master_dock_win']
app = display_output['app']
out_items = display_output['out_items']

# _display_short_long_firing_rate_index_comparison

In [ ]:
from pyphocorehelpers.programming_helpers import IPythonHelpers

long_short_fr_indicies_analysis_results = curr_active_pipeline.global_computation_results.computed_data['long_short_fr_indicies_analysis']
x_frs_index, y_frs_index = long_short_fr_indicies_analysis_results['x_frs_index'], long_short_fr_indicies_analysis_results['y_frs_index'] # use the all_results_dict as the computed data value
active_context = long_short_fr_indicies_analysis_results['active_context']
# IPythonHelpers.cell_vars(lambda: globals())

In [ ]:
long_short_fr_indicies_df = pd.DataFrame({'aclu': list(x_frs_index.keys()), 'replay_fr_idx': list(x_frs_index.values()), 'laps_fr_idx': list(y_frs_index.values())}) # , columns=['aclu','replay_frs']
long_short_fr_indicies_df.set_index('aclu', inplace=True)
# long_short_fr_indicies_df
# long_short_fr_indicies_df.to_clipboard()
long_short_fr_indicies_df

In [ ]:
long_short_fr_indicies_analysis_results

In [ ]:
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import build_neurons_color_map # for plot_short_v_long_pf1D_comparison

# Get existing long/short decoders from the cell under "# 2023-02-24 Decoders"
long_decoder, short_decoder = deepcopy(long_one_step_decoder_1D), deepcopy(short_one_step_decoder_1D)
assert np.all(long_decoder.xbin == short_decoder.xbin)

## 2023-05-01 - Test my interneuron hypothesis by looking at interneurons across the long/short divide

 1. Interneuron identity should be best determinant of active environment.

 2. Do people know anything about interneurons that activate in a given environment? For example, to interneurons activate alongside pyramidal cells when replaying a given environment?

 3. 


In [ ]:
# global_results.pf1D_dt.reset()
# global_results.pf2D_dt.reset()
# global_results.all_attributes

list(curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis.keys())

curr_active_pipeline.global_computation_results.computed_data.long_short

In [ ]:


# 1. Interneuron identity should be best determinant of active environment.

# 2. Do people know anything about interneurons that activate in a given environment? For example, to interneurons activate alongside pyramidal cells when replaying a given environment?

# 3. 
# global_results.pf1D_dt.reset()
# global_results.pf2D_dt.reset()
# global_results.all_attributes

list(curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis.keys())

curr_active_pipeline.global_computation_results.computed_data.long_short

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import JonathanFiringRateAnalysisResult

jonathan_firing_rate_analysis_result = JonathanFiringRateAnalysisResult(**curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis.to_dict())

# jonathan_firing_rate_analysis_result.neuron_replay_stats_df.to_clipboard()

In [ ]:
jonathan_firing_rate_analysis_result.rdf.rdf

In [ ]:
jonathan_firing_rate_analysis_result.time_binned_instantaneous_unit_specific_spike_rate.time_bins #.shape

In [ ]:

%matplotlib qt
instantaneous_fr_df = deepcopy(jonathan_firing_rate_analysis_result.time_binned_instantaneous_unit_specific_spike_rate.instantaneous_unit_specific_spike_rate_values)
instantaneous_fr_df['t'] = jonathan_firing_rate_analysis_result.time_binned_instantaneous_unit_specific_spike_rate.time_bins
instantaneous_fr_df #.plot()

smoothed_instantaneous_fr_df = instantaneous_fr_df.ewm(span=60).mean()


smoothed_instantaneous_fr_df.plot(x='t')
plt.title("Instantaneous Firing Rates")
plt.xlabel('time')
plt.ylabel('Firing Rate (Hz)')

In [ ]:
instantaneous_fr_df.set_index('t').to_clipboard()

In [ ]:
## Require placefield presence on either the long or the short
jonathan_firing_rate_analysis_result.neuron_replay_stats_df = jonathan_firing_rate_analysis_result.neuron_replay_stats_df[np.logical_or(jonathan_firing_rate_analysis_result.neuron_replay_stats_df['has_long_pf'], jonathan_firing_rate_analysis_result.neuron_replay_stats_df['has_short_pf'])]

In [ ]:
jonathan_firing_rate_analysis_result.neuron_replay_stats_df[np.logical_or(jonathan_firing_rate_analysis_result.neuron_replay_stats_df['has_long_pf'], jonathan_firing_rate_analysis_result.neuron_replay_stats_df['has_short_pf'])].index.to_numpy()

# 2023-05-19 - Testing S-only emergence, L-only replays in S, peak position remappings 

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import JonathanFiringRateAnalysisResult

jonathan_firing_rate_analysis_result = JonathanFiringRateAnalysisResult(**curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis.to_dict())
neuron_replay_stats_df = jonathan_firing_rate_analysis_result.neuron_replay_stats_df.copy()
neuron_replay_stats_df

In [ ]:

from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import SplitPartitionMembership, SetPartition
## 2023-05-19 - Get S-only pfs
is_S_pf_only = np.logical_and(np.logical_not(neuron_replay_stats_df['has_long_pf']), neuron_replay_stats_df['has_short_pf'])
is_S_only = neuron_replay_stats_df.track_membership == SplitPartitionMembership.RIGHT_ONLY
assert (is_S_pf_only == is_S_only).all()
S_only_aclus = neuron_replay_stats_df.index[is_S_only].to_numpy()
neuron_replay_stats_df[is_S_pf_only]


In [ ]:

## Show L-only pfs stop replaying on S
is_L_pf_only = np.logical_and(np.logical_not(neuron_replay_stats_df['has_short_pf']), neuron_replay_stats_df['has_long_pf'])
is_L_only = neuron_replay_stats_df.track_membership == SplitPartitionMembership.LEFT_ONLY
assert (is_L_pf_only == is_L_only).all()
L_only_aclus = neuron_replay_stats_df.index[is_L_only].to_numpy()
neuron_replay_stats_df[is_L_only]


## For ('kdiba', 'gor01', 'one', '2006-6-09_1-22-43') - Have L-only cells [24, 98] that have ['short_num_replays'] = [8, 7]. We were hoping that there would be few to no replays on the S-track that involved L-only cells.

In [ ]:
curr_context

In [ ]:
jonathan_firing_rate_analysis_result.neuron_replay_stats_df.to_clipboard()

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import compute_evening_morning_parition

difference_sorted_aclus, evening_sorted_aclus, morning_sorted_aclus = compute_evening_morning_parition(neuron_replay_stats_df, debug_print=True)
sorted_neuron_replay_stats_df = neuron_replay_stats_df.reindex(difference_sorted_aclus).copy() # This seems to work to re-sort the dataframe by the sort indicies
sorted_neuron_replay_stats_df

In [ ]:
from pyphocorehelpers.indexing_helpers import partition

# Split the replay stats based on the neuron types:
unique_included_neuron_types, (pyr_neuron_replay_stats_df, contaminated_neuron_replay_stats_df, interneuron_neuron_replay_stats_df) = partition(jonathan_firing_rate_analysis_result.neuron_replay_stats_df, 'neuron_type')
pyr_neuron_replay_stats_df

In [ ]:
from neuropy.core.neurons import NeuronType

%matplotlib qt
# Look at replays during ripples vs. those not during ripples. Also potentially PBEs vs. not PBEs.

# NeuronType.from_any_string_series(['pyr','intr'])
'pyr','cont','intr'

active_identifying_session_ctx = curr_active_pipeline.sess.get_context() # 'bapun_RatN_Day4_2019-10-15_11-30-06'
curr_active_pipeline.display('_display_jonathan_interactive_replay_firing_rate_comparison', active_identifying_session_ctx, included_neuron_types=NeuronType.from_any_string_series(['pyr'])) # only the pyramidal neurons

In [ ]:
curr_active_pipeline.reload_default_display_functions()

In [ ]:
curr_active_pipeline.display('_display_jonathan_interactive_replay_firing_rate_comparison', active_identifying_session_ctx, included_neuron_types=NeuronType.from_any_string_series(['intr']), require_placefield=False) # only the pyramidal neurons

In [ ]:
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import build_replays_custom_scatter_markers, CustomScatterMarkerMode

_curr_included_aclus = jonathan_firing_rate_analysis_result.neuron_replay_stats_df[np.logical_or(jonathan_firing_rate_analysis_result.neuron_replay_stats_df['has_long_pf'], jonathan_firing_rate_analysis_result.neuron_replay_stats_df['has_short_pf'])].index.to_numpy()
_curr_output = curr_active_pipeline.display('_display_batch_pho_jonathan_replay_firing_rate_comparison', active_identifying_session_ctx, included_unit_neuron_IDs=_curr_included_aclus, marker_split_mode=CustomScatterMarkerMode.NoSplit)

## 2023-04-13 - Shuffled Surprise
""" 
Relevant Functions:
`perform_full_session_leave_one_out_decoding_analysis`:
	`perform_leave_one_aclu_out_decoding_analysis`:	from pyphoplacecellanalysis.Analysis.Decoder.decoder_result import perform_leave_one_aclu_out_decoding_analysis
	`_analyze_leave_one_out_decoding_results`: from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.DefaultComputationFunctions import _analyze_leave_one_out_decoding_results
"""


In [ ]:
# 2023-04-18 - Refactored into decoder_result
result, result_df, result_df_grouped = long_results_obj.result, long_results_obj.result_df, long_results_obj.result_df_grouped

In [ ]:
curr_active_pipeline.global_computation_results

In [ ]:
from pyphoplacecellanalysis.Analysis.Decoder.decoder_result import DiagnosticDistanceMetricFigure

## Render the internactive slider that allows selecting the timebin index to debug
n_timebins = np.sum(long_results_obj.all_epochs_num_epoch_time_bins)
timebinned_neuron_info = long_results_obj.timebinned_neuron_info
result = long_results_obj.new_result
active_fig_obj, update_function = DiagnosticDistanceMetricFigure.build_interactive_diagnostic_distance_metric_figure(long_results_obj, timebinned_neuron_info, result)
active_fig_obj.integer_slider(n_timebins=n_timebins, update_func=update_function)

In [ ]:

# 1. Get set of cells active in a given time bin, for each compute the surprise of its placefield with the leave-one-out decoded posterior.

# 2. From the remainder of cells (those not active), randomly choose one to grab the placefield of and compute the surprise with that and the same posterior.

# Expectation: The cells that are included in the time bin are expected to have a lower surprise (be less correlated with) the posterior.

In [ ]:
from pyphoplacecellanalysis.Pho2D.PyQtPlots.Extensions.pyqtgraph_helpers import build_pyqtgraph_epoch_indicator_regions # Add session indicators to pyqtgraph plot
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import _helper_add_long_short_session_indicator_regions

from pyphocorehelpers.print_helpers import generate_html_string
# Long Short
# TODO 2023-04-18 - Can Refactor in terms of `plot_long_short_any_values`?

import pyphoplacecellanalysis.External.pyqtgraph as pg
# 'time_bin_indices': valid_time_bin_indicies, 'posterior_to_pf_mean_surprise': one_left_out_posterior_to_pf_surprises_mean, 'posterior_to_scrambled_pf_mean_surprise': one_left_out_posterior_to_scrambled_pf_surprises_mean}
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import plot_long_short_expected_vs_observed_firing_rates
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import plot_long_short_surprise_difference_plot

In [ ]:
win, plots = plot_long_short_surprise_difference_plot(curr_active_pipeline, long_results_obj, short_results_obj, long_epoch_name, short_epoch_name)


In [ ]:
long_results_obj
short_results_obj

In [ ]:
win, plots_tuple, legend = plot_long_short_expected_vs_observed_firing_rates(long_results_obj, short_results_obj, limit_aclus=list(S_only_aclus))
long_epoch_indicator_region_items, short_epoch_indicator_region_items = _helper_add_long_short_session_indicator_regions(win, long_epoch=curr_active_pipeline.filtered_epochs[long_epoch_name], short_epoch=curr_active_pipeline.filtered_epochs[short_epoch_name])

In [ ]:
long_results_obj


In [ ]:
a_plot = self.plots['difference__long']  # PlotDataItem 
a_curve = a_plot.curve # PlotCurveItem 

In [ ]:
a_plot.sigPointsClicked
# a_plot.sigPointsHovered

In [ ]:
curve.curve.setClickable(True)

In [ ]:
# Will make all plots clickable
clickedPen = pg.mkPen('#DDD', width=2)
lastClicked = []
def _test_scatter_plot_clicked(plot, points):
	global lastClicked
	for p in lastClicked:
		p.resetPen()
	print("clicked points", points)
	for p in points:
		p.setPen(clickedPen)
	lastClicked = points

main_scatter_clicked_connection = a_plot.sigClicked.connect(_test_scatter_plot_clicked)

In [ ]:
lastClicked, clickedPen, (main_scatter_hovered_connection, main_scatter_clicked_connection) = _helper_make_scatterplot_clickable(a_plot)

In [ ]:
# Need to map from time_bin_indicies to times
a_plot = self.plots['difference'] # PlotDataItem 
# a_plot.setLa
# win.graphicsItem().setLabel(axis='left', text='Normal v. Random - Surprise (Custom)')

# # Set the plot title with a LaTeX formula
# title = pg.LabelItem(justify='center')
# title.setText(r'<font size="4">JSD Surprise Diff: $\fn{JSD}{\matr{pf}_{i}, \Pr{\vec{x}_{t}|\vec{n}_{t}}} - \fn{JSD}{\matr{pf}_{j}, \Pr{\vec{x}_{t}|\vec{n}_{t}}}$</font>')
# win.addItem(title)

# win.graphicsItem().setLabel(axis='top', text=r'$\fn{JSD}{\matr{pf}_{i}, \Pr{\vec{x}_{t}|\vec{n}_{t}}} - \fn{JSD}{\matr{pf}_{j}, \Pr{\vec{x}_{t}|\vec{n}_{t}}}$')

In [ ]:
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import export_pyqtgraph_plot # works pretty well seemingly

export_pyqtgraph_plot(win)
# pg.setConfigOption('leftTitle', 'MathText')
# win.setTitle(r'JSD(p_x_given_n, pf[<font size="4"><b><span style="color:red;">i</span></b></font>]) - JSD(p_x_given_n, pf[<font size="4"><b>j</b></font>]) where <font size="4"><b>j</b></font> non-firing')

# win.setTitle(generate_html_string(f'JSD(p_x_given_n, pf[{i_str}]) - JSD(p_x_given_n, pf[{j_str}]) where {j_str} non-firing', font_size=8))

# r'$\fn{JSD}{\matr{pf}_{i}, \Pr{\vec{x}_{t}|\vec{n}_{t}}} - \fn{JSD}{\matr{pf}_{j}, \Pr{\vec{x}_{t}|\vec{n}_{t}}}$'
# title_str

In [ ]:
from pyqtgraph.GraphicsScene import exportDialog

# active_item = active_fig_obj.win
active_item = active_fig_obj.plot_dict['curr_cell_pf_curve']['plot_item']
exportDialog = exportDialog.ExportDialog(active_item.scene())
exportDialog.show(active_item)

In [ ]:
active_fig_obj.plot_dict['curr_cell_pf_curve']['plot_item']

In [ ]:
curr_active_pipeline.filtered_epochs[short_epoch_name].t_stop

In [ ]:
curr_active_pipeline.filtered_epochs[long_epoch_name].start_end_times

## Pre 2023-04-13

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib qt

from pyphoplacecellanalysis.Analysis.Decoder.decoder_result import plot_kourosh_activity_style_figure

from neuropy.core.neurons import NeuronType
# # Include only pyramidal aclus:
# print(f'all shared_aclus: {len(shared_aclus)}\nshared_aclus: {shared_aclus}')
# shared_aclu_neuron_type = long_session.neurons.neuron_type[np.isin(long_session.neurons.neuron_ids, shared_aclus)]
# assert len(shared_aclu_neuron_type) == len(shared_aclus)
# # Find only the aclus that are pyramidal:
# is_shared_aclu_pyramidal = (shared_aclu_neuron_type == NeuronType.PYRAMIDAL)
# pyramidal_only_shared_aclus = shared_aclus[is_shared_aclu_pyramidal]
# print(f'num pyramidal_only_shared_aclus: {len(pyramidal_only_shared_aclus)}\npyramidal_only_shared_aclus: {pyramidal_only_shared_aclus}')


## Drop Pyramidal but don't use only shared aclus:
all_aclus = deepcopy(long_session.neurons.neuron_ids)
neuron_type = long_session.neurons.neuron_type
assert len(neuron_type) == len(all_aclus)
# Find only the aclus that are pyramidal:
is_aclu_pyramidal = (neuron_type == NeuronType.PYRAMIDAL)
pyramidal_only_all_aclus = all_aclus[is_aclu_pyramidal]
print(f'num pyramidal_only_all_aclus: {len(pyramidal_only_all_aclus)}\npyramidal_only_all_aclus: {pyramidal_only_all_aclus}')

is_aclu_interneuron = (neuron_type == NeuronType.INTERNEURONS)
interneuron_only_all_aclus = all_aclus[is_aclu_interneuron]
print(f'num interneuron_only_all_aclus: {len(interneuron_only_all_aclus)}\ninterneurons: {interneuron_only_all_aclus}')

# app, win, plots, plots_data = plot_kourosh_activity_style_figure(long_results_obj, long_session, shared_aclus, epoch_idx=5, callout_epoch_IDXs=[0,1,2,3], skip_rendering_callouts=True)
# app, win, plots, plots_data = plot_kourosh_activity_style_figure(long_results_obj, long_session, pyramidal_only_shared_aclus, epoch_idx=2, callout_epoch_IDXs=[0,4], skip_rendering_callouts=False)
app, win, plots, plots_data = plot_kourosh_activity_style_figure(long_results_obj, long_session, pyramidal_only_all_aclus, epoch_idx=6, callout_epoch_IDXs=[2,4,6], skip_rendering_callouts=False)

In [ ]:

# Use the Jupyter Index Thing
from pyphoplacecellanalysis.Analysis.Decoder.decoder_result import DiagnosticDistanceMetricFigure

## Render the internactive slider that allows selecting the timebin index to debug
n_timebins = np.sum(long_results_obj.all_epochs_num_epoch_time_bins)
timebinned_neuron_info = long_results_obj.timebinned_neuron_info
result = long_results_obj.new_result
active_fig_obj, update_function = DiagnosticDistanceMetricFigure.build_interactive_diagnostic_distance_metric_figure(long_results_obj, timebinned_neuron_info, result)
active_fig_obj.integer_slider(n_timebins=n_timebins, update_func=update_function)

In [ ]:
active_fig_obj.export()

In [ ]:
app, win, self.plots, self.plots_data = plot_kourosh_activity_style_figure(long_results_obj, long_session, pyramidal_only_all_aclus, epoch_idx=11, callout_epoch_IDXs=[0,1,2, 3, 4, 5], skip_rendering_callouts=False)

# 2023-04-13 Show Surprise 

In [ ]:
import pyphoplacecellanalysis.External.pyqtgraph as pg
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.MultiContextComparingDisplayFunctions import plot_long_short, plot_long_short_any_values, plot_long_short_expected_vs_observed_firing_rates, _helper_add_long_short_session_indicator_regions
# plot_long_short(long_results_obj, short_results_obj)

In [ ]:
plot_long_short_any_values(long_results_obj=long_results_obj, short_results_obj=short_results_obj)

In [ ]:
plot_long_short_expected_vs_observed_firing_rates(long_results_obj=long_results_obj, short_results_obj=short_results_obj, limit_aclus=[89]) # 4, 89, 28, 97

In [ ]:
x_fn = lambda a_results_obj: a_results_obj.all_epochs_decoded_epoch_time_bins_mean[:,0]
# y_fn = lambda a_results_obj: a_results_obj.all_epochs_all_cells_one_left_out_posterior_to_scrambled_pf_surprises_mean
y_fn = lambda a_results_obj: a_results_obj.all_epochs_all_cells_one_left_out_posterior_to_pf_surprises_mean
# y_fn = lambda a_results_obj: a_results_obj.all_epochs_computed_one_left_out_posterior_to_pf_surprises

# (time_bins, neurons), (epochs, neurons), (epochs)
# all_epochs_computed_one_left_out_posterior_to_pf_surprises, all_epochs_computed_cell_one_left_out_posterior_to_pf_surprises_mean, all_epochs_all_cells_one_left_out_posterior_to_pf_surprises_mean
win, plots_tuple, legend = plot_long_short_any_values(long_results_obj, short_results_obj, x=x_fn, y=y_fn, limit_aclus=[20])

# 2023-04-13 - Find Good looking epochs:

In [21]:
import matplotlib.pyplot as plt
%matplotlib qt

from pyphocorehelpers.plotting.figure_management import PhoActiveFigureManager2D
from pyphoplacecellanalysis.GUI.Qt.Mixins.PaginationMixins import PaginatedFigureController
from pyphoplacecellanalysis.Pho2D.stacked_epoch_slices import DecodedEpochSlicesPaginatedFigureController

active_identifying_session_ctx = curr_active_pipeline.sess.get_context()
_out_pagination_controller = DecodedEpochSlicesPaginatedFigureController.init_from_decoder_data(long_results_obj.active_filter_epochs, long_results_obj.all_included_filter_epochs_decoder_result, 
	xbin=long_results_obj.original_1D_decoder.xbin, global_pos_df=global_session.position.df, a_name='TestDecodedEpochSlicesPaginationController', active_context=active_identifying_session_ctx,  max_subplots_per_page=10)
# _out_pagination_controller

active_identifying_ctx_string: "kdiba|gor01|one|2006-6-08_14-26-15|plot_single_epoch_slice|shared|1of14|[0 1 2 3 4 5 6 7 8 9]"


active_identifying_ctx_string: "kdiba|gor01|one|2006-6-08_14-26-15|plot_single_epoch_slice|shared|2of14|[10 11 12 13 14 15 16 17 18 19]"
active_identifying_ctx_string: "kdiba|gor01|one|2006-6-08_14-26-15|plot_single_epoch_slice|shared|3of14|[20 21 22 23 24 25 26 27 28 29]"
active_identifying_ctx_string: "kdiba|gor01|one|2006-6-08_14-26-15|plot_single_epoch_slice|shared|4of14|[30 31 32 33 34 35 36 37 38 39]"
active_identifying_ctx_string: "kdiba|gor01|one|2006-6-08_14-26-15|plot_single_epoch_slice|shared|5of14|[40 41 42 43 44 45 46 47 48 49]"
active_identifying_ctx_string: "kdiba|gor01|one|2006-6-08_14-26-15|plot_single_epoch_slice|shared|6of14|[50 51 52 53 54 55 56 57 58 59]"
active_identifying_ctx_string: "kdiba|gor01|one|2006-6-08_14-26-15|plot_single_epoch_slice|shared|7of14|[60 61 62 63 64 65 66 67 68 69]"
active_identifying_ctx_string: "kdiba|gor01|one|2006-6-08_14-26-15|plot_single_epoch_slice|shared|8of14|[70 71 72 73 74 75 76 77 78 79]"
active_identifying_ctx_string: "kdiba|gor

In [39]:
# Make sure the replays don't occur during the laps.
# from_PortionInterval,

## Get both as PortionIntervals to check for overlaps:
replays = curr_active_pipeline.sess.replay.epochs.to_PortionInterval()
laps = curr_active_pipeline.sess.laps.as_epoch_obj().to_PortionInterval() #.epochs.to_PortionInterval()
# laps

print(f'len(replays): {len(replays)}, len(laps): {len(laps)}, len(laps.difference(replays)): {len(laps.difference(replays))}')

len(replays): 403, len(laps): 77, len(laps.difference(replays)): 103


In [ ]:
_out_pagination_controller.params.debug_print = True
_pagination_widget = _out_pagination_controller.ui.mw.ui.paginator_controller_widget
_pagination_widget

In [ ]:
# .

current_page_idx = _out_pagination_controller.ui.mw.ui.paginator_controller_widget.current_page_idx
curr_page_data_indicies, curr_page_data_items =  _out_pagination_controller.plots_data.paginator.get_page_data(page_idx=current_page_idx)
print(f'current_page_idx: {current_page_idx}, curr_page_data_indicies: {curr_page_data_indicies}')


In [ ]:
_out_pagination_controller.on_paginator_control_widget_jump_to_page(3)

In [ ]:
from pyphocorehelpers.indexing_helpers import safe_find_index_in_list

def _get_current_page_data_indicies():
	""" captures `ui.mw.ui.paginator_controller_widget.current_page_idx`, `plots_data.paginator` """
	current_page_idx = _out_pagination_controller.ui.mw.ui.paginator_controller_widget.current_page_idx
	curr_page_data_indicies, curr_page_data_items =  _out_pagination_controller.plots_data.paginator.get_page_data(page_idx=current_page_idx)
	return current_page_idx, curr_page_data_indicies

In [ ]:
posterior_containers = filter_epochs_decoder_result.marginal_x_list.copy()

[a_marginal_x_list.p_x_given_n for a_marginal_x_list in filter_epochs_decoder_result.marginal_x_list]
[a_marginal_x_list.most_likely_positions_1D for a_marginal_x_list in filter_epochs_decoder_result.marginal_x_list]

posterior_containers
# posterior_containers = np.array(filter_epochs_decoder_result.marginal_x_list.copy())

In [ ]:
included_page_data_indicies, (curr_page_active_filter_epochs, curr_page_epoch_labels, curr_page_time_bin_containers, curr_page_posterior_containers) = epoch_slices_paginator.get_page_data(page_idx=1)
# included_page_data_indicies, (curr_page_active_filter_epochs, curr_page_epoch_labels, curr_page_time_bin_containers) = epoch_slices_paginator.get_page_data(page_idx=1)
included_page_data_indicies
curr_page_posterior_containers[0].p_x_given_n


In [ ]:
from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import DecodedFilterEpochsResult
from pyphocorehelpers.indexing_helpers import safe_len

# {k:safe_len(v) for k,v in filter_epochs_decoder_result.__dict__.items() if (safe_len(v) or 0) > 0}

## Only get the items that are num_filter_epochs long:
epoch_splittable_items = {k:v for k,v in filter_epochs_decoder_result.__dict__.items() if (safe_len(v) or 0) == filter_epochs_decoder_result.num_filter_epochs}
# epoch_splittable_items
epoch_splittable_item_names = list(epoch_splittable_items.keys())
# print(epoch_splittable_item_names)
# ('most_likely_positions_list', 'p_x_given_n_list', 'marginal_x_list', 'marginal_y_list', 'most_likely_position_indicies_list', 'spkcount', 'nbins', 'time_bin_containers', 'time_bin_edges')
# epoch_splittable_item_values = [epoch_splittable_items[k] for k in ('most_likely_positions_list', 'p_x_given_n_list', 'marginal_x_list', 'marginal_y_list', 'most_likely_position_indicies_list', 'nbins', 'time_bin_containers', 'time_bin_edges')]
epoch_splittable_item_values = [epoch_splittable_items[k] for k in ('epoch_description_list','time_bin_containers','marginal_x_list')]
# epoch_splittable_item_values

In [ ]:
# [23, 27, 29, ]
[16, 17, 18, 20, 21, 22, 23, 25, 26, 29]


# 2023-03-28 - Playing around with older computations/visualizations from the `_display_short_long_firing_rate_index_comparison` era:


2023-04-20 - Encountered issue with the replays in session '2006-6-08_14-26-15' where they are duplicated exactly twice, like the first half of the rows are legitimate entries and the second half are directly repeated versions of the first with the only difference appearing to be the 'epoch_id' column changes from 1 to 2. 'rel_id' column seems incorrect but different for some reason. It must be how the MATLAB script exports the values.

Also when I'm looking at only the `short_session.replay` there are many non-2 'epoch_id' values, which is strange. 

TODO: It could have something to do with Jonathan's code maybe? Because the 'replay_r' and 'replay_p' columns he added are different. SEEMS FALSE. It's this way even without running Jonathan's code, although the values might have been saved later?
	Also flat_replay_idx jumps from 689 to 1087 at the transition from epoch_id 1 to 2

UPDATE: the 'replay_r' and 'replay_p' columns aren't from Joanthan, they're in the original .replay_info.mat that's imported!


# Plot long|short firing rate index using 'long_short_fr_indicies_analyses':

In [ ]:
# fig_save_parent_path = Path(r'E:\Dropbox (Personal)\Active\Kamran Diba Lab\Results from 2023-01-20 - LongShort Firing Rate Indicies')
## Get the output path (active_session_figures_out_path) for this session (and all of its filtered_contexts as well):
active_identifying_session_ctx = curr_active_pipeline.sess.get_context() # 'bapun_RatN_Day4_2019-10-15_11-30-06'
figures_parent_out_path = create_daily_programmatic_display_function_testing_folder_if_needed()
active_session_figures_out_path = session_context_to_relative_path(figures_parent_out_path, active_identifying_session_ctx)
print(f'curr_session_parent_out_path: {active_session_figures_out_path}')
active_session_figures_out_path.mkdir(parents=True, exist_ok=True) # make folder if needed
curr_active_pipeline.display('_display_short_long_firing_rate_index_comparison', curr_active_pipeline.sess.get_context(), fig_save_parent_path=active_session_figures_out_path)
# plt.close() # closes the current figure

In [ ]:
curr_active_pipeline.display('_display_short_long_firing_rate_index_comparison', long_short_fr_indicies_analysis_results['active_context'], fig_save_parent_path=None)


In [ ]:
import matplotlib
# configure backend here
# matplotlib.use('Qt5Agg')
# backend_qt5agg
matplotlib.use('Qt5Agg') # non-interactive backend ## 2022-08-16 - Surprisingly this works to make the matplotlib figures render only to .png file, not appear on the screen!
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import _plot_long_short_firing_rate_indicies


# Plot long|short firing rate index:
long_short_fr_indicies_analysis_results = curr_active_pipeline.global_computation_results.computed_data['long_short_fr_indicies_analysis']
x_frs_index, y_frs_index = long_short_fr_indicies_analysis_results['x_frs_index'], long_short_fr_indicies_analysis_results['y_frs_index'] # use the all_results_dict as the computed data value
active_context = long_short_fr_indicies_analysis_results['active_context']
fig, _temp_full_fig_save_path = _plot_long_short_firing_rate_indicies(x_frs_index, y_frs_index, active_context, fig_save_parent_path=None, debug_print=False)
fig.show()

In [ ]:
ax = plt.gca()


In [ ]:
ax.spines[['top', 'right']].set_visible(True)

In [ ]:
# curr_active_pipeline.perform_specific_computation(computation_functions_name_whitelist=['_perform_long_short_firing_rate_analyses'], fail_on_exception=True, debug_print=False) # fail_on_exception MUST be True or error handling is all messed up 
long_short_fr_indicies_analysis_results = curr_active_pipeline.global_computation_results.computed_data['long_short_fr_indicies_analysis']
x_frs_index, y_frs_index = long_short_fr_indicies_analysis_results['x_frs_index'], long_short_fr_indicies_analysis_results['y_frs_index'] # use the all_results_dict as the computed data value
active_context = long_short_fr_indicies_analysis_results['active_context']
long_short_fr_indicies_analysis_results

# Other Programmatic Figures

In [ ]:
batch_extended_programmatic_figures(curr_active_pipeline=curr_active_pipeline)

In [ ]:
batch_programmatic_figures(curr_active_pipeline=curr_active_pipeline)

In [ ]:
# Output the 2D placefields for my presentation
from neuropy.core.neuron_identities import PlotStringBrevityModeEnum
from neuropy.plotting.ratemaps import BackgroundRenderingOptions

In [ ]:

# '_display_2d_placefield_result_plot_ratemaps_2D'
# %matplotlib inline
%matplotlib qt
# curr_active_pipeline.display('_display_2d_placefield_result_plot_ratemaps_2D', long_epoch_context) # MatplotlibRenderPlots


In [ ]:

# long_pf2D.plot_occupancy()
_out = short_one_step_decoder_2D.pf.plot_ratemaps_2D(included_unit_neuron_IDs=[2,4,5], bg_rendering_mode=BackgroundRenderingOptions.EMPTY, use_special_overlayed_title=False, missing_aclu_string_formatter=None, debug_print=False, brev_mode=PlotStringBrevityModeEnum.NONE)

## Single column output: subplots=(None, 9)


In [ ]:
# %%capture
active_display_to_pdf_fn(curr_active_pipeline, curr_display_function_name='_display_1d_placefields', debug_print=False) # 🟢✅ Now seems to be working and saving to PDF!! Still using matplotlib.use('Qt5Agg') mode and plots still appear.

In [ ]:
long_shared_aclus_only_decoder.pf.plot_occupancy()

In [ ]:
%matplotlib qt
long_one_step_decoder_2D.pf.plot_occupancy()

In [ ]:
long_one_step_decoder_2D.pf.plot_ratemaps_2D()

In [ ]:
# curr_active_pipeline.display('_display_short_long_pf1D_comparison', active_session_configuration_context=global_epoch_context, single_figure=False)

short_one_step_decoder_1D

In [ ]:
%matplotlib qt
active_identifying_session_ctx = curr_active_pipeline.sess.get_context() # 'bapun_RatN_Day4_2019-10-15_11-30-06'

long_single_cell_pfmap_processing_fn = None
short_single_cell_pfmap_processing_fn = None

# long_single_cell_pfmap_processing_fn = lambda i, aclu, pfmap: 0.5 * pfmap # flip over the y-axis
# short_single_cell_pfmap_processing_fn = lambda i, aclu, pfmap: -0.5 * pfmap # flip over the y-axis

# pad = 1
# long_single_cell_pfmap_processing_fn = lambda i, aclu, pfmap: (0.5 * pfmap) + (0.5*pad) # shift the baseline up by half
# short_single_cell_pfmap_processing_fn = lambda i, aclu, pfmap: (-0.5 * pfmap * pad) + (0.5*pad) # flip over the y-axis, shift the baseline down by half

# pad = 1
# long_single_cell_pfmap_processing_fn = lambda i, aclu, pfmap: (0.5 * pfmap * pad) + (0.5*pad) # shift the baseline up by half
# short_single_cell_pfmap_processing_fn = lambda i, aclu, pfmap: (0.5 * pfmap * pad) + (0.5*pad) # flip over the y-axis, shift the baseline down by half
# long_single_cell_pfmap_processing_fn = lambda i, aclu, pfmap: (0.5 * pfmap * pad) # shift the baseline up by half
# short_single_cell_pfmap_processing_fn = lambda i, aclu, pfmap: (0.5 * pfmap * pad) # flip over the y-axis, shift the baseline down by half


# long_single_cell_pfmap_processing_fn = lambda i, aclu, pfmap: (1.0 * pfmap * pad) # shift the baseline up by half
# short_single_cell_pfmap_processing_fn = lambda i, aclu, pfmap: (-1.0 * pfmap * pad) + (1.0*pad) # this does not work and results in short being fully filled. I think this is because the fill_between gets reversed since everything is below baseline

sort_idx = None

out = curr_active_pipeline.display('_display_short_long_pf1D_comparison', active_identifying_session_ctx, single_figure=False, debug_print=False, fignum='Short v Long pf1D Comparison',
                                   long_kwargs={'sortby': sort_idx, 'single_cell_pfmap_processing_fn': long_single_cell_pfmap_processing_fn},
                                   short_kwargs={'sortby': sort_idx, 'single_cell_pfmap_processing_fn': short_single_cell_pfmap_processing_fn, 'curve_hatch_style': {'hatch':'///', 'edgecolor':'k'}},
                                  )

# ax = out.axes[0]

In [ ]:
sort_idx = np.argsort(product_overlap_scalars_df.prod_overlap.to_numpy())[::-1] # the `[::-1]` term reverses the array, which by defaul is returned in ascending order and we want descending
sort_idx

In [ ]:


curr_ratemap = long_one_step_decoder_1D.pf.ratemap
curr_ratemap.get_sort_indicies()
# .pf1D.ratemap

In [ ]:
# Create `master_dock_win` - centralized plot output window to collect individual figures/controls in (2022-08-18)

active_identifying_session_ctx = curr_active_pipeline.filtered_contexts['maze']
display_output = curr_active_pipeline.display('_display_context_nested_docks', active_identifying_session_ctx, enable_gui=False, debug_print=True) # returns {'master_dock_win': master_dock_win, 'app': app, 'out_items': out_items}
master_dock_win = display_output['master_dock_win']
app = display_output['app']
out_items = display_output['out_items']

# def _get_curr_figure_format_config():
# 	""" Aims to fetch the current figure_format_config and context from the figure_format_config widget:    
# 	Implicitly captures: `out_items`, `active_config_name`, `active_identifying_filtered_session_ctx` 
# 	"""
# 	## Get the figure_format_config from the figure_format_config widget:
# 	# Fetch the context from the GUI:
# 	_curr_gui_session_ctx, _curr_gui_out_display_items = out_items[active_config_name]
# 	_curr_gui_figure_format_config_widget = _curr_gui_out_display_items[active_identifying_filtered_session_ctx.adding_context('display_fn', display_fn_name='figure_format_config_widget')] # [0] is seemingly not needed to unpack the tuple
# 	if _curr_gui_figure_format_config_widget is not None:
# 		# has GUI for config
# 		figure_format_config = _curr_gui_figure_format_config_widget.figure_format_config
# 	else:
# 		# has non-GUI provider of figure_format_config
# 		figure_format_config = _curr_gui_figure_format_config_widget.figure_format_config

# 	if debug_print:
# 		print(f'recovered gui figure_format_config: {figure_format_config}')

# 	return figure_format_config

# figure_format_config = _get_curr_figure_format_config()

# 2023-04-27 - Idea: Candidate Replay "Quality" Metric
Observed that I visually distinguish "good" replays from bad ones based on their mostly-monotonically increasing nature.

#### They don't have to:
	Span the whole track
	Start or end at an end-cap
	Increase linearly

#### The algorithm must:
	tolerate ocasional jumps in an otherwise linear sequence
	allow sequences to "start" only halfway through. They should extract out the coherent sequence

#### Ideas:
	- just a linear fit
	- a monotonicity check using some sort of cumulative sum over the differences in position.
	- "radon transform"?

# 2023-05-02 - Session Validation Check Info
Generate info about the number of laps detected, the duration, the number of cells, the number of replays, etc for the active session so that it can be ensured that there wasn't an error that is messing up the analysis.

In [ ]:
curr_active_pipeline.active_sess_config

# Pipeline Comprehensive:


In [ ]:
## Top level objects:
curr_active_pipeline.active_sess_config
curr_active_pipeline.sess



# Function of config names or contexts:
curr_active_pipeline.filtered_contexts

curr_active_pipeline.filtered_sessions
curr_active_pipeline.active_configs # each is a `InteractivePlaceCellConfig` type object
	curr_active_pipeline._stage.active_configs['maze1']



curr_active_pipeline.computation_results


curr_active_pipeline.global_computation_results



In [ ]:
from neuropy.core.epoch import EpochsAccessor, Epoch

sess = curr_active_pipeline.sess
# sess.replay.epochs.debug_print_info('replays')


def debug_print_session_epochs_info(sess, session_name:str):
	""" 

	Example:
		debug_print_session_epochs_info(curr_active_pipeline.sess, 'sess')
		debug_print_session_epochs_info(long_session, 'long_session')
		debug_print_session_epochs_info(short_session, 'short_session')
		debug_print_session_epochs_info(global_session, 'global_session')

	sess:
		num replays: 1729
		num laps: 80
	long_session:
		num replays: 661
		num laps: 40
	short_session:
		num replays: 1068
		num laps: 40
	global_session:
		num replays: 1729
		num laps: 80
		
	
	"""	

	replays_df = sess.replay
	laps_df = sess.laps.as_epoch_obj().to_dataframe()
	print(f'{session_name}:')
	print(f'\tnum replays: {replays_df.epochs.n_epochs}')
	print(f'\tnum laps: {laps_df.epochs.n_epochs}')

# (long_one_step_decoder_1D, short_one_step_decoder_1D), (long_one_step_decoder_2D, short_one_step_decoder_2D) = compute_short_long_constrained_decoders(curr_active_pipeline, recalculate_anyway=True)
long_epoch_name, short_epoch_name, global_epoch_name = curr_active_pipeline.find_LongShortGlobal_epoch_names()
long_epoch_context, short_epoch_context, global_epoch_context = [curr_active_pipeline.filtered_contexts[a_name] for a_name in (long_epoch_name, short_epoch_name, global_epoch_name)]
long_session, short_session, global_session = [curr_active_pipeline.filtered_sessions[an_epoch_name] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
# long_results, short_results, global_results = [curr_active_pipeline.computation_results[an_epoch_name]['computed_data'] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
# long_pf1D, short_pf1D, global_pf1D = long_results.pf1D, short_results.pf1D, global_results.pf1D
# long_pf2D, short_pf2D, global_pf2D = long_results.pf2D, short_results.pf2D, global_results.pf2D
# decoding_time_bin_size = long_one_step_decoder_1D.time_bin_size # 1.0/30.0 # 0.03333333333333333



debug_print_session_epochs_info(curr_active_pipeline.sess, 'sess')
debug_print_session_epochs_info(long_session, 'long_session')
debug_print_session_epochs_info(short_session, 'short_session')
debug_print_session_epochs_info(global_session, 'global_session')


# epoch_df = sess.laps.to_dataframe()

# epoch_df.epochs.n_epochs


# epoch_name = 'laps'
# num_epochs = epoch_df.epochs.n_epochs # np.shape(epoch_df)[0]
# print(f'num {epoch_name}: {num_epochs}')

# sess.replay.to_dataframe()

# def debug_print_session_epochs_info(num_updated_total_items: int, num_original_total_items: int, item_label=None, subsession_name=None):
#     print('{}/{} total {} remain in subsession {}'.format(num_updated_total_items, num_original_total_items, (item_label or "items"), (subsession_name or "")))

# debug_print_subsession_epochs_differences(num_updated_total_items=len(long_session.replay), num_original_total_items=len(long_session.replay_backup), item_label='replays', subsession_name='"long"')
# debug_print_subsession_epochs_differences(num_updated_total_items=len(short_session.replay), num_original_total_items=len(short_session.replay_backup), item_label='replays', subsession_name='"short"')



In [ ]:

def debug_print_subsession_epochs_differences(num_updated_total_items: int, num_original_total_items: int, item_label=None, subsession_name=None):
    print('{}/{} total {} remain in subsession {}'.format(num_updated_total_items, num_original_total_items, (item_label or "items"), (subsession_name or "")))

debug_print_subsession_epochs_differences(num_updated_total_items=len(long_session.replay), num_original_total_items=len(long_session.replay_backup), item_label='replays', subsession_name='"long"')
debug_print_subsession_epochs_differences(num_updated_total_items=len(short_session.replay), num_original_total_items=len(short_session.replay_backup), item_label='replays', subsession_name='"short"')


In [ ]:
debug_print_subsession_epochs_differences(num_updated_total_items=len(long_session.laps), num_original_total_items=len(long_session.replay_backup), item_label='laps', subsession_name='"long"')
debug_print_subsession_epochs_differences(num_updated_total_items=len(short_session.laps), num_original_total_items=len(short_session.replay_backup), item_label='laps', subsession_name='"short"')


In [ ]:
# Session Comprehensive:
sess = curr_active_pipeline.sess

sess.
# sess.panel_dataframes_overview()

## Visual:
# SyntaxError('invalid syntax', ('<string>', 1, 1, '<bound method DataSession.panel_dataframes_overview of DataSession(2006-6-07_16-40-19.xml)>'))
# pn.extension(comms='ipywidgets')

In [ ]:
before_num_rows = self.n_epochs        
filtered_epochs = convert_PortionInterval_to_epochs_df(_convert_start_end_tuples_list_to_PortionInterval(zip(self.starts, self.stops)))
after_num_rows = np.shape(filtered_epochs)[0]
changed_num_rows = after_num_rows - before_num_rows
print(f'Dataframe Changed from {before_num_rows} -> {after_num_rows} ({changed_num_rows = })')

In [ ]:
from neuropy.utils.debug_helpers import debug_print_placefield, debug_print_spike_counts, debug_print_subsession_neuron_differences, print_aligned_columns
from neuropy.utils.misc import print_seconds_human_readable

debug_print_spike_counts(global_session)

In [ ]:
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import _compare_computation_results
# curr_active_pipeline.computation_results
# pf_neurons_diff = _compare_computation_results(curr_active_pipeline.computation_results.maze1_PYR, curr_active_pipeline.computation_results.maze2_PYR)
pf_neurons_diff = _compare_computation_results(long_results, short_results)
pf_neurons_diff

In [ ]:
print(pf_neurons_diff)

In [ ]:

from neuropy.analyses.laps import estimate_session_laps

curr_laps = global_session.laps
curr_laps.to_dataframe()

### Plotting Validations

In [ ]:
%matplotlib qt
short_one_step_decoder_1D.pf.plot_ratemaps_1D()

In [ ]:
short_results.pf1D_Decoder.pf.plot_ratemaps_1D()

In [ ]:
short_results.pf2D_Decoder.pf.plot_ratemaps_2D()
short_one_step_decoder_2D.pf.plot_ratemaps_2D()

In [ ]:
%matplotlib qt
long_session.plot_laps_2d()

In [ ]:
from pyphoplacecellanalysis.PhoPositionalData.plotting.laps import plot_laps_2d

%matplotlib qt
fig, out_axes_list = plot_laps_2d(global_session, legacy_plotting_mode=False)
out_axes_list[0].set_title('New Pho Position Thresholding Estimated Laps')
fig.canvas.manager.set_window_title('New Pho Position Thresholding Estimated Laps')

In [ ]:
## Laps 
from pyphoplacecellanalysis.GUI.PyVista.InteractivePlotter.Mixins.LapsVisualizationMixin import LapsVisualizationMixin

curr_position_df, lap_specific_position_dfs = LapsVisualizationMixin._compute_laps_specific_position_dfs(curr_active_pipeline.sess)
lap_specific_position_dfs = [curr_position_df.groupby('lap').get_group(i)[['t','x','y','lin_pos']] for i in curr_active_pipeline.sess.laps.lap_id] # dataframes split for each ID:
laps_position_times_list = [np.squeeze(lap_pos_df[['t']].to_numpy()) for lap_pos_df in lap_specific_position_dfs]
laps_position_traces_list = [lap_pos_df[['x','y']].to_numpy().T for lap_pos_df in lap_specific_position_dfs]
## Build Epochs:
epochs = curr_active_pipeline.sess.laps.to_dataframe()
epoch_slices = epochs[['start', 'stop']].to_numpy()
epoch_description_list = [f'lap {epoch_tuple.lap_id} (maze: {epoch_tuple.maze_id}, direction: {epoch_tuple.lap_dir})' for epoch_tuple in epochs[['lap_id','maze_id','lap_dir']].itertuples()]
# print(f'epoch_description_list: {epoch_description_list}') # epoch_descriptions: ['lap 41 (maze: 2, direction: 1)', 'lap 42 (maze: 2, direction: 0)', 'lap 43 (maze: 2, direction: 1)', 'lap 44 (maze: 2, direction: 0)', 'lap 45 (maze: 2, direction: 1)', 'lap 46 (maze: 2, direction: 0)', 'lap 47 (maze: 2, direction: 1)', 'lap 48 (maze: 2, direction: 0)', 'lap 49 (maze: 2, direction: 1)', 'lap 50 (maze: 2, direction: 0)', 'lap 51 (maze: 2, direction: 1)', 'lap 52 (maze: 2, direction: 0)', 'lap 53 (maze: 2, direction: 1)', 'lap 54 (maze: 2, direction: 0)', 'lap 55 (maze: 2, direction: 1)', 'lap 56 (maze: 2, direction: 0)', 'lap 57 (maze: 2, direction: 1)', 'lap 58 (maze: 2, direction: 0)', 'lap 59 (maze: 2, direction: 1)', 'lap 60 (maze: 2, direction: 0)', 'lap 61 (maze: 2, direction: 1)', 'lap 62 (maze: 2, direction: 0)', 'lap 63 (maze: 2, direction: 1)', 'lap 64 (maze: 2, direction: 0)', 'lap 65 (maze: 2, direction: 1)', 'lap 66 (maze: 2, direction: 0)', 'lap 67 (maze: 2, direction: 1)', 'lap 68 (maze: 2, direction: 0)', 'lap 69 (maze: 2, direction: 1)', 'lap 70 (maze: 2, direction: 0)', 'lap 71 (maze: 2, direction: 1)', 'lap 72 (maze: 2, direction: 0)', 'lap 73 (maze: 2, direction: 1)', 'lap 74 (maze: 2, direction: 0)', 'lap 75 (maze: 2, direction: 1)', 'lap 76 (maze: 2, direction: 0)', 'lap 77 (maze: 2, direction: 1)', 'lap 78 (maze: 2, direction: 0)', 'lap 79 (maze: 2, direction: 1)']
# -